<a href="https://colab.research.google.com/github/reesha-rsh/MLb4/blob/main/Homework/Homework_4__pawpularity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 4 - Pawpularity

It's time to train your first model on ~cats~ real data. The dataset of cute pets is avaialble here -  [Pawpularity Contest - Predict the popularity of shelter pet photos](https://www.kaggle.com/c/petfinder-pawpularity-score/data)


Your task is to predict engagement with a pet's profile based on the photograph for that profile. You are also provided with hand-labelled metadata for each photo. The dataset for this competition therefore comprises both images and tabular data.

In this homework we'll only use tabular meta data. Our goal is to fit linear reggresion on binary features. Let's see if we can do it better than predicting a constant.
![PetFinder.my - Pawpularity Contest](https://www.petfinder.my/images/cuteness_meter-showcase.jpg)

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize':(14,9)})
%matplotlib inline

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/MLb4/Homework/Homework_4__pawpularity/train.csv") # you can downloand only train.csv from https://www.kaggle.com/c/petfinder-pawpularity-score/data

In [ ]:
data

Let's create train/test split. We keep 20% of data for testing model performance

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2, random_state=42, stratify=data.Pawpularity)
train.shape, test.shape

In [ ]:
# Check that you have the same train/test split as othersф
assert(train.iloc[5].Pawpularity == 13)
assert(train.iloc[-5].Face == 1)
assert(test.iloc[5].Group == 1)
assert(test.iloc[-5].Pawpularity == 39)

### Baseline

In [ ]:
from sklearn.metrics import mean_squared_error

Create a copy for test dataset:

In [ ]:
test = test.copy()

Set baseline based on mean/median/mode:

In [ ]:
test['baseline'] = 0 # try to use mean/median/mode as baseline

In [ ]:
mean_squared_error(test.Pawpularity, test.baseline, squared=False)

### Regression

Let's try to train linear regression on binary features

In [ ]:
features = data.columns[1:-1]
features

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso

In [ ]:
model = LinearRegression() # Compare several regression models, select the best. Don't forget to tune hyper params

In [ ]:
# Train your model here:
# model.fit()
# test['pred'] = model.predict()

Calculate score:

In [ ]:
mean_squared_error(test.Pawpularity, test.pred, squared=False)

Check params:

In [ ]:
list(zip(features, model.coef_)) # model's weights

In [ ]:
model.intercept_ # model's bias

Analyze errors:

In [ ]:
## True value / pred value plot:
#sns.scatterplot(x=test.pred, y=test.Pawpularity)

In [ ]:
## Errors distribution:
#sns.histplot(test.Pawpularity - test.pred)

**Compare several regression models (L1, L2 regularization), select the best. Don't forget to tune hyperparams**

Print your best score and what type of regression you used to achieve it:

### Polinomial Regression
Do polynomial features improve the score?

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures

[make_pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.make_pipeline.html) helps to combine several steps into one model

In [ ]:
model = make_pipeline(
    ...
)

In [ ]:
# Hint: to check wieght use:
# list(zip(t.get_feature_names(input_features=features), model.steps[1][1].coef_)) # model's weights

Print your best score with pollinomial reggresion:

### Feature selection

Let's use Lasso for feature selection. Sklearn's [SelectFromModel](https://scikit-learn.org/stable/modules/feature_selection.html#feature-selection-using-selectfrommodel) will help

In [ ]:
from sklearn.feature_selection import SelectFromModel ##

In [ ]:
model = make_pipeline(
    ...
)

Print your best score with feature selection: